# Bloque 3 · Combinando recuperación y generación

En este bloque daremos el siguiente paso del flujo RAG: pasaremos de recuperar fragmentos (Bloque 2) a generar respuestas completas usando la API de RAG (`retrieve_and_generate`) de Amazon Bedrock. Analizaremos cómo el modelo genera texto a partir del contexto recuperado y practicaremos la personalización del *prompt*.

## ¿En qué se diferencia del Bloque 2?

- **Bloque 2**: nos enfocamos en recuperar fragmentos relevantes mediante el endpoint `Retrieve` y aprendimos a parsear la respuesta.
- **Bloque 3**: incorporamos un modelo generativo. Pedimos a Bedrock que, además de recuperar contextos, redacte una respuesta siguiendo nuestras instrucciones.

## LLM

### ¿Qué es un LLM?

Un **LLM (Large Language Model, Modelo de Lenguaje Grande)** es un tipo de modelo de inteligencia artificial entrenado con grandes volúmenes de texto para entender y generar lenguaje natural. Estos modelos aprenden patrones, relaciones semánticas y estructuras del lenguaje humano a partir de millones o billones de documentos.

Los LLMs pueden:
- **Comprender** texto en lenguaje natural (preguntas, instrucciones, contexto)
- **Generar** texto coherente y contextualmente relevante
- **Sintetizar** información de múltiples fuentes
- **Adaptar** el tono y estilo según las instrucciones recibidas

Ejemplos populares incluyen modelos como Claude (de Anthropic), GPT (de OpenAI), y Titan Text (de AWS), entre otros. En Amazon Bedrock podemos acceder a varios de estos modelos mediante una API unificada.

### Rol del LLM dentro de un RAG

- **Interpreta la pregunta** y detecta la intención.
- **Lee los fragmentos recuperados** para extraer evidencia.
- **Redacta** una respuesta final, aplicando el tono y formato especificado.
- **Cita fuentes** si se lo pedimos explícitamente en el prompt.

Sin un buen prompt, el LLM puede ignorar información relevante o inventar detalles. Por eso es fundamental controlar el mensaje que le enviamos.

El LLM aporta redacción fluida, síntesis y la capacidad de adaptar el tono al público. Nuestro trabajo consiste en suministrarle el contexto correcto y un *prompt* claro para reducir al mínimo las alucinaciones.

## API `retrieve_and_generate`

- Combina en una sola llamada la búsqueda en la knowledge base y la generación de texto.
- Permite indicar qué modelo usar (`modelArn`), cómo recuperar los fragmentos y cómo estructurar la generación (parámetros y *prompt template*).
- Devuelve tanto la respuesta generada como los fragmentos utilizados, para auditar y mostrar citas.

Trabajaremos directamente con esta API usando `boto3`.

In [ ]:
!pip install boto3

In [11]:
import boto3
from typing import Any, Dict

AWS_REGION = "us-west-2"
KNOWLEDGE_BASE_ID = "7DUKWTRFX3"
MODEL_ARN = "us.deepseek.r1-v1:0"

# Completar con las credenciales
session = boto3.Session(
aws_access_key_id='',
aws_secret_access_key='',
aws_session_token=''
)

cliente = session.client(
    "bedrock-agent-runtime",
    region_name=AWS_REGION
)

print("Cliente de Bedrock inicializados correctamente.")


Cliente de Bedrock inicializados correctamente.


## Diseño del prompt

Un *prompt template* típico para RAG incluye:
- Un mensaje al modelo sobre su rol (`Eres un asistente experto en ...`).
- Las instrucciones sobre cómo usar el contexto (`Solo responde usando la información proporcionada`).
- El espacio reservado para insertar los fragmentos (`{context}`) y la pregunta (`{query}`).
- Indicaciones de formato y tono.

La actividad práctica se centrará en modificar este template para observar cómo cambia la respuesta.

## Implementando el flujo completo de RAG

Para combinar recuperación y generación, necesitamos definir un **prompt template** que guíe al LLM sobre cómo usar los fragmentos recuperados. Este template actúa como un "manual de instrucciones" que le dice al modelo:

- **Su rol y contexto**: qué tipo de asistente es y para qué propósito
- **Cómo usar el contexto**: que debe basarse únicamente en los fragmentos recuperados
- **Qué hacer cuando falta información**: cómo manejar casos donde no hay contexto suficiente
- **Formato y tono**: cómo estructurar la respuesta

El template utiliza placeholders como `$query$` y `$search_results$` que Bedrock reemplaza automáticamente con la pregunta del usuario y los fragmentos recuperados de la knowledge base.

La función `generar_con_prompt()` encapsula toda la lógica técnica para invocar `retrieve_and_generate`:

1. **Configura la recuperación**: especifica cuántos fragmentos recuperar (`top_k`) mediante búsqueda vectorial
2. **Configura la generación**: define parámetros como `max_tokens` (límite de longitud) y `temperature` (control de aleatoriedad)
3. **Construye la solicitud**: ensambla la configuración completa con el knowledge base ID, modelo ARN y el prompt template
4. **Invoca la API**: realiza la llamada a `retrieve_and_generate` que ejecuta ambos pasos (recuperación y generación) en una sola operación

El resultado incluye tanto el texto generado como las citas que vinculan cada parte de la respuesta con los fragmentos fuente, permitiendo verificar la trazabilidad y confiabilidad de la información.

In [7]:
DEFAULT_PROMPT_TEMPLATE = """Eres un asistente educativo especializado en Retrieval-Augmented Generation (RAG) y sistemas de búsqueda semántica.
Tu rol es ayudar a los estudiantes a comprender conceptos relacionados con embeddings, búsqueda vectorial, chunking, similitud coseno y arquitecturas RAG.

Responde la pregunta del usuario usando ÚNICAMENTE la información de los resultados de búsqueda proporcionados del material del taller.

IMPORTANTE:
- Usa ÚNICAMENTE información de los resultados de búsqueda del material educativo
- Si no hay información suficiente en los resultados, responde: "No encontré una respuesta exacta en el material del taller disponible"
- No inventes ni fabriques información que no esté presente en los resultados
- Explica los conceptos de manera clara y educativa, ayudando a los estudiantes a entender los fundamentos de RAG

Pregunta del usuario:

$query$

Resultados de búsqueda:

$search_results$

$output_format_instructions$"""


def generar_con_prompt(
    pregunta: str,
    prompt_template: str = None,
    top_k: int = 4,
    max_tokens: int = 600,
    temperature: float = 0.2
) -> Dict[str, Any]:
    """
    Genera una respuesta usando retrieve_and_generate de Bedrock.

    Args:
        pregunta: La pregunta del usuario
        prompt_template: Template del prompt (opcional, usa DEFAULT_PROMPT_TEMPLATE si no se proporciona)
        top_k: Número de resultados a recuperar
        max_tokens: Máximo de tokens en la respuesta
        temperature: Controla la aleatoriedad/creatividad de la generación (0.0-1.0).
                     Valores bajos (0.1-0.3) producen respuestas más deterministas y precisas,
                     ideales para tareas que requieren exactitud. Valores altos (0.7-1.0) generan
                     respuestas más creativas y variadas. Para RAG educativo, valores bajos (0.2)
                     son recomendados para mantener precisión y coherencia con el contexto recuperado.

    Returns:
        Diccionario con la respuesta de la API
    """
    # Usar template por defecto si no se proporciona uno
    if prompt_template is None:
        prompt_template = DEFAULT_PROMPT_TEMPLATE

    # Configuración de recuperación
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": top_k
        }
    }

    # Configuración base
    config = {
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGE_BASE_ID,
            "modelArn": MODEL_ARN,
            "retrievalConfiguration": retrieval_config,
            "generationConfiguration": {
                "inferenceConfig": {
                    "textInferenceConfig": {
                        "maxTokens": max_tokens,
                        "temperature": temperature
                    }
                }
            }
        }
    }

    # Parámetros de la llamada
    params = {
        "input": {
            "text": pregunta
        },
        "retrieveAndGenerateConfiguration": config
    }

    # Realizar llamada a la API
    respuesta = cliente.retrieve_and_generate(**params)
    return respuesta


## Visualizando respuestas y trazabilidad de fuentes

Una vez que el modelo genera una respuesta usando `retrieve_and_generate`, necesitamos una forma de presentar tanto el contenido generado como su trazabilidad. La función `mostrar_generacion()` procesa la respuesta estructurada de la API para extraer y formatear esta información.

**Estructura de la respuesta de la API:**

La respuesta de `retrieve_and_generate` contiene:
- **`output.text`**: El texto completo generado por el modelo
- **`citations`**: Una lista de citas que vinculan partes específicas del texto generado con sus fuentes

Cada cita en `citations` tiene dos componentes principales:
- **`generatedResponsePart`**: Contiene el fragmento de texto generado que está siendo citado, junto con su posición exacta (`span`) en el texto completo (índices `start` y `end`)
- **`retrievedReferences`**: Una lista de referencias a los documentos fuente que respaldan esa parte de la respuesta. Cada referencia incluye:
  - La ubicación del documento (URI de S3)
  - Metadatos con información adicional sobre el origen
  - Un preview del contenido del fragmento fuente

**Procesamiento y visualización:**

La función `mostrar_generacion()` realiza tres tareas principales:

1. **Extrae y formatea el texto generado**: Presenta la respuesta completa con un ancho máximo de línea para facilitar la lectura
2. **Procesa las citas**: Para cada cita, muestra qué parte del texto está respaldada por fuentes y sus posiciones exactas
3. **Muestra las referencias**: Lista todas las fuentes documentales asociadas a cada cita, incluyendo un preview del contenido para verificar la relevancia

Esta visualización es fundamental para **auditar la calidad del RAG**: permite verificar que el modelo está usando correctamente el contexto recuperado, identificar qué fuentes respaldan cada afirmación, y detectar posibles alucinaciones cuando partes de la respuesta no tienen referencias asociadas.

In [ ]:
from textwrap import wrap
import json

def mostrar_generacion(respuesta: Dict[str, Any]) -> None:
    output = respuesta.get("output", {})
    texto = output.get("text", "<Sin respuesta>")

    print("Respuesta generada:\n")
    for linea in wrap(texto, width=100):
        print(linea)

    print("\n" + "="*100)
    print("Citas encontradas:")
    print("="*100)

    citas = respuesta.get("citations", [])
    if not citas:
        print("- No se devolvieron citas. Revisa el prompt o la coverage de la knowledge base.")
    else:
        texto_completo = texto  # Guardamos el texto completo para mostrar los spans

        for idx, cita in enumerate(citas, start=1):
            # Obtener la parte del texto citada
            generated_part = cita.get("generatedResponsePart", {})
            text_part = generated_part.get("textResponsePart", {})
            texto_citado = text_part.get("text", "")
            span = text_part.get("span", {})
            start = span.get("start", 0)
            end = span.get("end", len(texto_completo))

            print(f"\n--- Cita #{idx} ---")
            print(f"Texto citado (posiciones {start}-{end}):")
            # Mostrar el texto citado con un ancho limitado
            for linea in wrap(texto_citado, width=90):
                print(f"  {linea}")

            # Obtener las referencias recuperadas
            retrieved_refs = cita.get("retrievedReferences", [])

            if not retrieved_refs:
                print("  Referencias: Ninguna referencia recuperada para esta cita.")
            else:
                print(f"  Referencias ({len(retrieved_refs)}):")
                for ref_idx, ref in enumerate(retrieved_refs, start=1):
                    # Intentar obtener la URI desde location o metadata
                    location = ref.get("location", {})
                    metadata = ref.get("metadata", {})

                    # Priorizar metadata, luego location
                    fuente = (
                        metadata.get("x-amz-bedrock-kb-source-uri") or
                        location.get("s3Location", {}).get("uri") or
                        "Fuente desconocida"
                    )

                    print(f"    [{ref_idx}] {fuente}")

                    # Opcional: mostrar un preview del contenido citado
                    content = ref.get("content", {})
                    contenido_texto = content.get("text", "")
                    if contenido_texto:
                        preview = contenido_texto[:150].replace("\n", " ").strip()
                        if len(contenido_texto) > 150:
                            preview += "..."
                        print(f"        Preview: {preview}")


pregunta_inicial = "¿Qué métodos de búsqueda se usan para comparar vectores?"
prompt_personalizado = DEFAULT_PROMPT_TEMPLATE

respuesta_generada = generar_con_prompt(pregunta_inicial, prompt_personalizado)
print(json.dumps(respuesta_generada, indent=2))
mostrar_generacion(respuesta_generada)


## Actividad · Añadir nuevos documentos y modificar parámetros

Objetivo: demostrar cómo un flujo RAG puede incorporar conocimiento reciente sin reentrenar el modelo.

1. Hacé una pregunta sobre un documento **que todavía no esté** en la knowledge base.
2. Observá cómo responde el sistema (probablemente indicando que no tiene contexto suficiente).
3. Agregá el documento al data source y lanzá una nueva ingesta.
4. Volvé a hacer la pregunta y verifica cómo ahora el LLM puede responder basándose en la nueva información.
5. Jugá con el prompt y los parámetros de la API y fijate como cambia la respuesta.

In [ ]:
pregunta_nueva = "¿Cómo se prepara una foaccia?"
prompt_para_nuevo_doc = DEFAULT_PROMPT_TEMPLATE

respuesta = generar_con_prompt(pregunta_nueva, prompt_para_nuevo_doc)
mostrar_generacion(respuesta)


## Reflexión final

- Un *prompt* bien redactado guía al modelo para usar únicamente la evidencia recuperada.
- La API de RAG permite iterar rápidamente: añadir documentos y volver a preguntar sin reentrenar el modelo.
- La trazabilidad (citas) es clave para generar confianza en el chatbot.

En el siguiente bloque integraremos todo esto dentro de Chainlit para ofrecer una experiencia conversacional completa.